In [76]:
#https://zhuanlan.zhihu.com/p/59772104
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader
import torch.utils.data as data

train_x = [torch.Tensor([1, 1, 1, 1, 1, 1, 1]),
           torch.Tensor([2, 2, 2, 2, 2, 2]),
           torch.Tensor([3, 3, 3, 3, 3]),
           torch.Tensor([4, 4, 4, 4]),
           torch.Tensor([5, 5, 5]),
           torch.Tensor([6, 6]),
           torch.Tensor([7])
           ]

pad_x = rnn_utils.pad_sequence(train_x, batch_first=True)

print(pad_x)
class MyData(data.Dataset):
    def __init__(self, data_seq):
        self.data_seq = data_seq

    def __len__(self):
        return len(self.data_seq)

    def __getitem__(self, idx):
        return self.data_seq[idx]


if __name__=='__main__':
    #data = MyData(train_x)
    data = MyData(pad_x)
    data_loader = DataLoader(data, batch_size=2, shuffle=True)
    batch_x = iter(data_loader).next()
    print(batch_x)
    print('END')
    #以数据的最大长度，后两位都是00，在lstm中浪费两次计算。

tensor([[1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 0.],
        [3., 3., 3., 3., 3., 0., 0.],
        [4., 4., 4., 4., 0., 0., 0.],
        [5., 5., 5., 0., 0., 0., 0.],
        [6., 6., 0., 0., 0., 0., 0.],
        [7., 0., 0., 0., 0., 0., 0.]])
tensor([[6., 6., 0., 0., 0., 0., 0.],
        [5., 5., 5., 0., 0., 0., 0.]])
END


In [94]:
Linear=nn.Linear(7,7)#input_dim,out_dim
print(Linear(pad_x[2:4]))

tensor([[ 1.6631, -0.1358,  0.4446,  1.3461,  0.2082, -0.7027, -0.6471],
        [ 2.8690, -0.4374,  0.2876,  3.1230,  0.4198, -0.0236, -1.4215]],
       grad_fn=<AddmmBackward>)


In [77]:
#pad
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data

if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=2, shuffle=True, 
                             collate_fn=collate_fn)
    batch_x = iter(data_loader).next()
    print(batch_x)
    print('END')
    #以batch的最大长度为准，而不是整个数据的最大长度。

tensor([[4., 4., 4., 4.],
        [6., 6., 0., 0.]])
END


In [103]:
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data_length = [len(sq) for sq in data]
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data, data_length


if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=3, shuffle=True, 
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x, 
                                                  batch_x_len, batch_first=True)
print(batch_x,batch_x_len)
print(batch_x_pack)


tensor([[2., 2., 2., 2., 2., 2.],
        [4., 4., 4., 4., 0., 0.],
        [5., 5., 5., 0., 0., 0.]]) [6, 4, 3]
PackedSequence(data=tensor([2., 4., 5., 2., 4., 5., 2., 4., 5., 2., 4., 2., 2.]), batch_sizes=tensor([3, 3, 3, 2, 1, 1]))


In [110]:
Linear=nn.Linear(6,6)#input_dim,out_dim
print(Linear(batch_x))

tensor([[-0.8202,  0.9555, -1.8077, -0.6296,  2.7744,  0.6284],
        [ 0.3939,  1.6663, -1.6440, -0.2789,  3.8227,  1.0817],
        [-0.4596,  0.9900, -1.3149, -1.2184,  3.2859,  2.0111]],
       grad_fn=<AddmmBackward>)


In [120]:
x=batch_x_pack.data.reshape(-1,1)#the size -1 is inferred from other dimensions
print(x)
Linear=nn.Linear(1,1)#(input_dim,out_dim)
print(Linear(x))

tensor([[2.],
        [4.],
        [5.],
        [2.],
        [4.],
        [5.],
        [2.],
        [4.],
        [5.],
        [2.],
        [4.],
        [2.],
        [2.]])
tensor([[2.5511],
        [4.3836],
        [5.2998],
        [2.5511],
        [4.3836],
        [5.2998],
        [2.5511],
        [4.3836],
        [5.2998],
        [2.5511],
        [4.3836],
        [2.5511],
        [2.5511]], grad_fn=<AddmmBackward>)


In [80]:
#使其返回的数据符合 [batch, sequence_len, input_size],即batch_first
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data_length = [len(sq) for sq in data]
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data.unsqueeze(-1), data_length

if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=3, shuffle=True, 
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x, 
                                                  batch_x_len, batch_first=True)
    
print(batch_x,batch_x_len)
print(batch_x_pack)

tensor([[[3.],
         [3.],
         [3.],
         [3.],
         [3.]],

        [[6.],
         [6.],
         [0.],
         [0.],
         [0.]],

        [[7.],
         [0.],
         [0.],
         [0.],
         [0.]]]) [5, 2, 1]
PackedSequence(data=tensor([[3.],
        [6.],
        [7.],
        [3.],
        [6.],
        [3.],
        [3.],
        [3.]]), batch_sizes=tensor([3, 2, 1, 1, 1]))


In [89]:
if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=3, shuffle=True,
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x, batch_x_len, batch_first=True)

    net = nn.RNN(1, 10, 2,batch_first=True)

    print(batch_x_pack)
    h0 = torch.rand(2, 3, 10)
    out, (h1) = net(batch_x_pack,h0)
    print(out.data.shape)#第一位表示统计整个数据中共有多少个个非零的数据。即每个数据的RNN的输出。
    print(batch_x_pack.data.shape)
    print(out.batch_sizes)
    print(batch_x_pack.batch_sizes)
    print(out)
    print(h1.shape)#与h0的shape相同。
    out_pad, out_len = rnn_utils.pad_packed_sequence(out, batch_first=True)
    print(out_pad.shape)#第一位是句子数，第二位是时间步长（最长的句长），第三维是特征向量维度。
    print(out_len)#每一个句子的长度
    print(out_pad[1])#假如下标为1的句子真实长度为2，其余位（假如长度位3），共5位，那么out_pad[1]只有前两行有值，其余行为0（3行），
    #表示每一个特征映射为hidden_size，其余pad0.

PackedSequence(data=tensor([[2.],
        [6.],
        [7.],
        [2.],
        [6.],
        [2.],
        [2.],
        [2.],
        [2.]]), batch_sizes=tensor([3, 2, 1, 1, 1, 1]))
torch.Size([9, 10])
torch.Size([9, 1])
tensor([3, 2, 1, 1, 1, 1])
tensor([3, 2, 1, 1, 1, 1])
PackedSequence(data=tensor([[-0.7220, -0.4395, -0.4604, -0.0896,  0.4151,  0.2507,  0.3952, -0.4562,
          0.1178, -0.2781],
        [-0.8316, -0.6554, -0.5307,  0.1159,  0.7410,  0.4759,  0.6168, -0.8307,
          0.0897, -0.4659],
        [-0.8032, -0.7134, -0.3445,  0.1038,  0.7269,  0.4488,  0.8143, -0.9084,
          0.1758, -0.4301],
        [-0.0782,  0.3156, -0.4263,  0.1117, -0.0807,  0.2368,  0.0160,  0.0583,
          0.1945,  0.1365],
        [-0.0237, -0.0461, -0.6207,  0.0447,  0.0545,  0.4614,  0.0634, -0.5351,
          0.2593, -0.1730],
        [-0.2774,  0.0767, -0.4326,  0.1480,  0.1423,  0.3460,  0.3616, -0.1442,
          0.3477,  0.0010],
        [-0.2503,  0.1254, -0.4386,  0.0400, 

In [83]:
#完整
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader
import torch.utils.data as data


train_x = [torch.Tensor([1, 1, 1, 1, 1, 1, 1]),
           torch.Tensor([2, 2, 2, 2, 2, 2]),
           torch.Tensor([3, 3, 3, 3, 3]),
           torch.Tensor([4, 4, 4, 4]),
           torch.Tensor([5, 5, 5]),
           torch.Tensor([6, 6]),
           torch.Tensor([7])
           ]

class MyData(data.Dataset):
    def __init__(self, data_seq):
        self.data_seq = data_seq

    def __len__(self):
        return len(self.data_seq)

    def __getitem__(self, idx):
        return self.data_seq[idx]


def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data_length = [len(sq) for sq in data]
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data.unsqueeze(-1), data_length


if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=3, shuffle=True,
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x,
                                                  batch_x_len, batch_first=True)
    print(batch_x)
    print(batch_x_pack)
    net = nn.RNN(1, 10, 2, batch_first=True)
    h0 = torch.rand(2, 3, 10)
    #c0 = torch.rand(2, 3, 10)
    out, h1 = net(batch_x_pack, h0)
    out_pad, out_len = rnn_utils.pad_packed_sequence(out, batch_first=True)
    print(out)

tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[4.],
         [4.],
         [4.],
         [4.],
         [0.],
         [0.],
         [0.]],

        [[5.],
         [5.],
         [5.],
         [0.],
         [0.],
         [0.],
         [0.]]])
PackedSequence(data=tensor([[1.],
        [4.],
        [5.],
        [1.],
        [4.],
        [5.],
        [1.],
        [4.],
        [5.],
        [1.],
        [4.],
        [1.],
        [1.],
        [1.]]), batch_sizes=tensor([3, 3, 3, 2, 1, 1, 1]))
PackedSequence(data=tensor([[-0.1359,  0.1547, -0.0126,  0.3267, -0.3614,  0.4605, -0.3252,  0.4271,
          0.0496, -0.1898],
        [-0.2145, -0.0027, -0.0835,  0.5169, -0.3309,  0.7682, -0.4270,  0.4915,
          0.2825, -0.3199],
        [ 0.0034,  0.1260,  0.1975,  0.3381, -0.2281,  0.7027, -0.3199,  0.5543,
          0.3908, -0.4691],
        [ 0.0390,  0.0141,  0.1996,  0.3342,  0.3321,  0.7216, -0.3177,